In [6]:
import face_recognition
import cv2
import matplotlib.pyplot as plt
import os
from matplotlib import pyplot as plt
from PIL import Image
import torch

# تحميل موديل YOLO
model = torch.hub.load('ultralytics/yolov5', 'yolov5s')

# تحميل الصورة
img_path = 'test/10.jfif'  # تأكد من المسار الصحيح للصورة
img = cv2.imread(img_path)
if img is None:
    print("Error: Image not found!")
else:
    print("Image loaded successfully!")

# إجراء اكتشاف الأجسام باستخدام YOLO
results = model(img_path)

# التعامل مع النتائج: رسم المربعات حول الأجسام المكتشفة
results.render()  # رسم المربعات حول الأجسام المكتشفة

# عرض الصورة مع المربعات باستخدام matplotlib
plt.imshow(cv2.cvtColor(results.ims[0], cv2.COLOR_BGR2RGB))  # التأكد من تحويل الألوان لـ RGB
plt.axis('off')
plt.show()

# تحويل الصورة إلى RGB للعرض مع OpenCV
img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

# العثور على الوجوه في الصورة
face_locations = face_recognition.face_locations(img_rgb)
print(f"Faces found: {len(face_locations)}")
print(f"Face locations: {face_locations}")

# إعداد مجموعة البيانات والتعرف على الوجوه
train_path = os.path.join("dataset")  # مسار البيانات الخاصة بك
train_encodes = []
train_classes = []

# المرور عبر مجموعة الأشخاص في الداتا سيت
for person_name in os.listdir(train_path):
    person_folder = os.path.join(train_path, person_name)
    
    if not os.path.isdir(person_folder):
        continue  # تخطي لو مش مجلد

    for image_name in os.listdir(person_folder):
        image_path = os.path.join(person_folder, image_name)
        img_person = cv2.imread(image_path)
        img_person_rgb = cv2.cvtColor(img_person, cv2.COLOR_BGR2RGB)

        # استخراج أكواد الوجوه
        encodings = face_recognition.face_encodings(img_person_rgb)
        if encodings:
            train_encodes.append(encodings[0])  # حفظ الكود الخاص بالوجه
            train_classes.append(person_name)  # حفظ اسم الشخص

# اختبار الصورة: التعرف على كل وجه على حدة
if len(face_locations) > 0:
    test_encodings = face_recognition.face_encodings(img_rgb, face_locations)

    for (top, right, bottom, left), img_encoding in zip(face_locations, test_encodings):
        # مقارنة الأكواد المخزنة في الداتا سيت
        results = face_recognition.compare_faces(train_encodes, img_encoding)
        face_name = "Unknown"

        # إذا تم العثور على تطابق، احصل على الاسم
        face_distances = face_recognition.face_distance(train_encodes, img_encoding)
        if len(face_distances) > 0:
            best_match_index = face_distances.argmin()
            if results[best_match_index]:
                face_name = train_classes[best_match_index]

        # رسم المستطيل حول الوجه
        cv2.rectangle(img_rgb, (left, top), (right, bottom), (0, 255, 0), 2)

        # كتابة الاسم تحت المستطيل
        cv2.putText(img_rgb, face_name, (left, bottom + 20), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 0, 0), 2)

# عرض النتيجة مع الوجوه المكتشفة
plt.figure(figsize=(10, 8))
plt.axis("off")
plt.title(f"Detected Faces and Objects")
plt.imshow(img_rgb)
plt.show()


Using cache found in C:\Users\Lenovo/.cache\torch\hub\ultralytics_yolov5_master
YOLOv5  2025-5-8 Python-3.10.0 torch-2.7.0+cpu CPU

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients, 16.4 GFLOPs
Adding AutoShape... 
C:\Users\Lenovo/.cache\torch\hub\ultralytics_yolov5_master\models\common.py:906: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):


Image loaded successfully!
Faces found: 1
Face locations: [(38, 146, 113, 71)]
